In [1]:
from pathlib import Path
import sys
import re
import pandas as pd
import shutil
import os
from multiprocessing import Pool
import prody
import pickle
ABPRED_DIR = Path.cwd().parent
if ABPRED_DIR not in sys.path:
    sys.path.append(str(ABPRED_DIR))
from AbPred.FoldX import FoldX
from modeller import *
from modeller.optimizers import molecular_dynamics, conjugate_gradients
from modeller.automodel import autosched

In [23]:
# adapt benchmark dataframe
adapt_dataframe = pd.read_csv('../data/3bdy_adapt.csv',sep=";")
#ab_data_singleMut.sort_values('#Pdb',inplace=True)

PDBS_DIR = Path("../benchmark/adapt_bench/")

adapt_pdb = '../benchmark/adapt_bench/3bdy.protein.pdb'

In [45]:
adapt_dataframe = adapt_dataframe.loc[-adapt_dataframe.Mutation.str.contains('S30b|S30d')]

#  Run mutation protocol by modeller

In [46]:
amino3to1dict = {'ALA': 'A','CYS': 'C','ASP': 'D','GLU': 'E','PHE': 'F','GLY': 'G',
                 'HIS': 'H','ILE': 'I','LYS': 'K','LEU': 'L','MET': 'M','ASN': 'N',
                 'PRO': 'P','GLN': 'Q','ARG': 'R','SER': 'S','THR': 'T','VAL': 'V',
                 'TRP': 'W','TYR': 'Y'}

def convertAAcode(m_splits):
    """Function to get the 3letters aa in amino3to1dict
    using the split_mutations() output"""
    for k,v in amino3to1dict.items():
        if v == m_splits[0]:
            return k
        
def split_mutations(mut_str):
    """Function to split a [mutation] string, searching the mutaion patter and return groups.
    Output is a list with 3 elements [wt_aa,aa_number,mut_aa] """
    
    search_mut = re.search("([A-Z])([0-9]+[A-Z]*)([A-Z])",mut_str,flags=re.I)
    m_splits = search_mut.groups()
    
    return list(m_splits)

In [47]:
def mutate_model(modelname,respos,restyp,chain,random_seed):
    
    #
    #  mutate_model.py
    #
    #     Usage:   python mutate_model.py modelname respos resname chain > logfile
    #
    #     Example: python mutate_model.py 1t29 1699 LEU A > 1t29.log
    #
    #
    #  Creates a single in silico point mutation to sidechain type and at residue position
    #  input by the user, in the structure whose file is modelname.pdb
    #  The conformation of the mutant sidechain is optimized by conjugate gradient and
    #  refined using some MD.
    #
    #  Note: if the model has no chain identifier, specify "" for the chain argument.
    #


    def optimize(atmsel, sched):
        #conjugate gradient
        for step in sched:
            step.optimize(atmsel, max_iterations=200, min_atom_shift=0.001)
        #md
        refine(atmsel)
        cg = conjugate_gradients()
        cg.optimize(atmsel, max_iterations=200, min_atom_shift=0.001)


    #molecular dynamics
    def refine(atmsel):
        # at T=1000, max_atom_shift for 4fs is cca 0.15 A.
        md = molecular_dynamics(cap_atom_shift=0.39, md_time_step=10.0,
                                md_return='FINAL')
        init_vel = True
        for (its, equil, temps) in ((200, 20, (150.0, 250.0, 400.0, 700.0, 1000.0,1500.0)),
                                    (2000, 200,
                                     (1500.0,1000.0, 800.0, 600.0, 500.0, 400.0, 300.0))):
            for temp in temps:
                md.optimize(atmsel, init_velocities=init_vel, temperature=temp,
                             max_iterations=its, equilibrate=equil)
                init_vel = False


    #use homologs and dihedral library for dihedral angle restraints
    def make_restraints(mdl1, aln):
        rsr = mdl1.restraints
        rsr.clear()
        s = selection(mdl1)
        for typ in ('stereo', 'phi-psi_binormal'):
            rsr.make(s, restraint_type=typ, aln=aln, spline_on_site=True)
        for typ in ('omega', 'chi1', 'chi2', 'chi3', 'chi4'):
            rsr.make(s, restraint_type=typ+'_dihedral', spline_range=4.0,
                    spline_dx=0.3, spline_min_points = 5, aln=aln,
                    spline_on_site=True)

    #first argument
    #modelname, respos, restyp, chain, = sys.argv[1:]


    log.verbose()

    # Set a different value for rand_seed to get a different final model
    env = environ(rand_seed=random_seed)

    env.io.hetatm = False
    #soft sphere potential
    env.edat.dynamic_sphere=False
    #lennard-jones potential (more accurate)
    env.edat.dynamic_lennard=True
    env.edat.contact_shell = 4.0
    env.edat.update_dynamic = 0.39

    # Read customized topology file with phosphoserines (or standard one)
    env.libs.topology.read(file='$(LIB)/top_heav.lib')

    # Read customized CHARMM parameter library with phosphoserines (or standard one)
    env.libs.parameters.read(file='$(LIB)/par.lib')


    # Read the original PDB file and copy its sequence to the alignment array:
    mdl1 = model(env, file=modelname)
    ali = alignment(env)
    ali.append_model(mdl1, atom_files=modelname, align_codes=modelname)

    #set up the mutate residue selection segment
    s = selection(mdl1.chains[chain].residues[respos])

    #perform the mutate residue operation
    s.mutate(residue_type=restyp)
    #get two copies of the sequence.  A modeller trick to get things set up
    ali.append_model(mdl1, align_codes=modelname)

    # Generate molecular topology for mutant
    mdl1.clear_topology()
    mdl1.generate_topology(ali[-1])


    # Transfer all the coordinates you can from the template native structure
    # to the mutant (this works even if the order of atoms in the native PDB
    # file is not standard):
    #here we are generating the model by reading the template coordinates
    mdl1.transfer_xyz(ali)

    # Build the remaining unknown coordinates
    mdl1.build(initialize_xyz=False, build_method='INTERNAL_COORDINATES')

    #yes model2 is the same file as model1.  It's a modeller trick.
    mdl2 = model(env, file=modelname)

    #required to do a transfer_res_numb
    #ali.append_model(mdl2, atom_files=modelname, align_codes=modelname)
    #transfers from "model 2" to "model 1"
    mdl1.res_num_from(mdl2,ali)

    #It is usually necessary to write the mutated sequence out and read it in
    #before proceeding, because not all sequence related information about MODEL
    #is changed by this command (e.g., internal coordinates, charges, and atom
    #types and radii are not updated).

    mdl1.write(file=modelname+restyp+respos+'.tmp')
    mdl1.read(file=modelname+restyp+respos+'.tmp')

    #set up restraints before computing energy
    #we do this a second time because the model has been written out and read in,
    #clearing the previously set restraints
    make_restraints(mdl1, ali)

    #a non-bonded pair has to have at least as many selected atoms
    mdl1.env.edat.nonbonded_sel_atoms=1

    sched = autosched.loop.make_for_model(mdl1)

    #only optimize the selected residue (in first pass, just atoms in selected
    #residue, in second pass, include nonbonded neighboring atoms)
    #set up the mutate residue selection segment
    s = selection(mdl1.chains[chain].residues[respos])

    mdl1.restraints.unpick_all()
    mdl1.restraints.pick(s)

    s.energy()

    s.randomize_xyz(deviation=4.0)

    mdl1.env.edat.nonbonded_sel_atoms=2
    optimize(s, sched)

    #feels environment (energy computed on pairs that have at least one member
    #in the selected)
    mdl1.env.edat.nonbonded_sel_atoms=1
    optimize(s, sched)

    s.energy()

    dope_energy = mdl1.assess_normalized_dope()
    #give a proper name
    mdl1.write(file=modelname[:-4]+"."+chain+respos+amino3to1dict[restyp]+'.mut.pdb')

    #delete the temporary file
    os.remove(modelname+restyp+respos+'.tmp')
    
    return dope_energy


In [48]:
def mainModmut(row):
    pdbid = '3bdy'
    partners = 'LH_V'
    mutations = row.Mutation
    pdbname = '{}.protein.pdb'.format(pdbid)
    
    # get argument for mutate function
    modelfile = PDBS_DIR/Path(pdbname)
    mutation_split = split_mutations(mutations)
    # wildtype res
    restyp = convertAAcode(mutation_split[-1])
    respos = mutation_split[1]
    chain = row.Chain
    
    #print(modelfile,restyp,chain,respos)
    #return
    dope_energy =  mutate_model(modelname=str(modelfile.resolve()),restyp=restyp,chain=chain,respos=respos,random_seed=10)
    return dope_energy

In [50]:
CWD = Path.cwd()

energy_mut = list()
try:
    #os.chdir('skempiAB-modeller')
    with Pool(12) as p:
        dope_energy = p.map(mainModmut, [row for i,row in adapt_dataframe.iterrows()])
        energy_mut.append(dope_energy)
finally:
    os.chdir(CWD)


                         MODELLER 9.20, 2018/05/30, r11208

     PROTEIN STRUCTURE MODELLING BY SATISFACTION OF SPATIAL RESTRAINTS


                     Copyright(c) 1989-2018 Andrej Sali
                            All Rights Reserved

                             Written by A. Sali
                               with help from
              B. Webb, M.S. Madhusudhan, M-Y. Shen, G.Q. Dong,
          M.A. Marti-Renom, N. Eswar, F. Alber, M. Topf, B. Oliva,
             A. Fiser, R. Sanchez, B. Yerkovich, A. Badretdinov,
                     F. Melo, J.P. Overington, E. Feyfant
                 University of California, San Francisco, USA
                    Rockefeller University, New York, USA
                      Harvard University, Cambridge, USA
                   Imperial Cancer Research Fund, London, UK
              Birkbeck College, University of London, London, UK


Kind, OS, HostName, Kernel, Processor: 4, Darwin Mac-Pro-de-Victor.local 14.5.0 x86_64
Date and time of compi




                         MODELLER 9.20, 2018/05/30, r11208

     PROTEIN STRUCTURE MODELLING BY SATISFACTION OF SPATIAL RESTRAINTS


                     Copyright(c) 1989-2018 Andrej Sali
                            All Rights Reserved

                             Written by A. Sali
                               with help from
              B. Webb, M.S. Madhusudhan, M-Y. Shen, G.Q. Dong,
          M.A. Marti-Renom, N. Eswar, F. Alber, M. Topf, B. Oliva,
             A. Fiser, R. Sanchez, B. Yerkovich, A. Badretdinov,
                     F. Melo, J.P. Overington, E. Feyfant
                 University of California, San Francisco, USA
                    Rockefeller University, New York, USA
                      Harvard University, Cambridge, USA
                   Imperial Cancer Research Fund, London, UK
              Birkbeck College, University of London, London, UK


Kind, OS, HostName, Kernel, Processor: 4, Darwin Mac-Pro-de-Victor.local 14.5.0 x86_64
Date and time of com






Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       253994     248.041     0.242


openf___224_> Open           ${MODINSTALL9v20}/modlib/af_mnchdef.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib

openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/af_mnchdef.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/af_mnchdef.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/af_mnchdef.lib
openf___224_> Open        

read_to_681_> topology.submodel read from topology file:        3
rdrrwgh_268_> Number of residue types:       21
openf___224_> Open           $(LIB)/top_heav.lib
read_to_681_> topology.submodel read from topology file:        3
openf___224_> Open           $(LIB)/par.lib
openf___224_> Open           $(LIB)/par.lib
openf___224_> Open           $(LIB)/par.lib

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:       393430     384.209     0.375
Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:       393430     384.209     0.375

openf___224_> Open           $(LIB)/par.lib

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:       393430     384.209     0.375openf___224_> Open           $(LIB)/par.lib

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:       393430     384.209     0.375

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:       395030     385.771     0.377openf___224_> Open           $(LIB)/par.lib

Dynamically allocated m



                              227     561     661     112       0
openf___224_> Open           /Users/vfica/Documents/GitHub/Master-thesis/benchmark/adapt_bench/3bdy.protein.pdb


openf___224_> Open           /Users/vfica/Documents/GitHub/Master-thesis/benchmark/adapt_bench/3bdy.protein.pdb
read_pa_232_> parameters    BONDS   ANGLS  DIHEDS IMPROPS    MODE

Dynamically allocated memory at    amaxsequence [B,KiB,MiB]:       442603     432.229     0.422
Dynamically allocated memory at    amaxsequence [B,KiB,MiB]:       442603     432.229     0.422
Dynamically allocated memory at amaxcoordinates [B,KiB,MiB]:       442527     432.155     0.422
Dynamically allocated memory at    amaxsequence [B,KiB,MiB]:       442603     432.229     0.422
Dynamically allocated memory at amaxcoordinates [B,KiB,MiB]:       442527     432.155     0.422openf___224_> Open           /Users/vfica/Documents/GitHub/Master-thesis/benchmark/adapt_bench/3bdy.protein.pdb
                              227     561     66

...

17 Sidechain Chi_4 dihedral restraints:       0       0      0   0.000   0.000      0.0000       1.000
18 Disulfide distance restraints      :       0       0      0   0.000   0.000      0.0000       1.000
19 Disulfide angle restraints         :       0       0      0   0.000   0.000      0.0000       1.000
20 Disulfide dihedral angle restraints:       0       0      0   0.000   0.000      0.0000       1.000
21 Lower bound distance restraints    :       0       0      0   0.000   0.000      0.0000       1.000
22 Upper bound distance restraints    :       0       0      0   0.000   0.000      0.0000       1.000
23 Distance restraints 3 (SDCH-MNCH)  :       0       0      0   0.000   0.000      0.0000       1.000
24 Sidechain Chi_5 dihedral restraints:       0       0      0   0.000   0.000      0.0000       1.000
25 Phi/Psi pair of dihedral restraints:       3       0      0  27.259  27.259     0.57662E-03   1.000
26 Distance restraints 4 (SDCH-SDCH)  :       0       0      0   0.000   

18 Disulfide distance restraints      :       0       0      0   0.000   0.000      0.0000       1.000
19 Disulfide angle restraints         :       0       0      0   0.000   0.000      0.0000       1.000
20 Disulfide dihedral angle restraints:       0       0      0   0.000   0.000      0.0000       1.000
21 Lower bound distance restraints    :       0       0      0   0.000   0.000      0.0000       1.000
22 Upper bound distance restraints    :       0       0      0   0.000   0.000      0.0000       1.000
23 Distance restraints 3 (SDCH-MNCH)  :       0       0      0   0.000   0.000      0.0000       1.000
24 Sidechain Chi_5 dihedral restraints:       0       0      0   0.000   0.000      0.0000       1.000
25 Phi/Psi pair of dihedral restraints:       3       0      0  27.873  27.873    -0.20258       1.000
26 Distance restraints 4 (SDCH-SDCH)  :       0       0      0   0.000   0.000      0.0000       1.000
27 Distance restraints 5 (X-Y)        :       0       0      0   0.000   

Atomic shift for contacts update (UPDATE_DYNAMIC) :    0.390
LENNARD_JONES_SWITCH                              :    6.500   7.500
COULOMB_JONES_SWITCH                              :    6.500   7.500
RESIDUE_SPAN_RANGE                                :        0   99999
NLOGN_USE                                         :       15
CONTACT_SHELL                                     :    4.000
DYNAMIC_PAIRS,_SPHERE,_COULOMB,_LENNARD,_MODELLER :        T       F       F       T       F
SPHERE_STDV                                       :    0.050
RADII_FACTOR                                      :    0.820
Current energy                                    :          24.0776





Summary of the restraint violations: 

   NUM     ... number of restraints.
   NUMVI   ... number of restraints with RVIOL > VIOL_REPORT_CUT[i].
   RVIOL   ... relative difference from the best value.
   NUMVP   ... number of restraints with -Ln(pdf) > VIOL_REPORT_CUT2[i].
   RMS_1   ... RMS(feature, minimally_violated_

Number of all, selected real atoms                :     4079    4079
Number of all, selected pseudo atoms              :        0       0
Number of all static, selected restraints         :    20134      64
COVALENT_CYS                                      :        F
NONBONDED_SEL_ATOMS                               :        1
Number of non-bonded pairs (excluding 1-2,1-3,1-4):   769545
Dynamic pairs routine                             : 1, NATM x NATM double loop
Atomic shift for contacts update (UPDATE_DYNAMIC) :    0.390
LENNARD_JONES_SWITCH                              :    6.500   7.500
COULOMB_JONES_SWITCH                              :    6.500   7.500
RESIDUE_SPAN_RANGE                                :        1    9999
NLOGN_USE                                         :       15
CONTACT_SHELL                                     :   15.000
DYNAMIC_PAIRS,_SPHERE,_COULOMB,_LENNARD,_MODELLER :        T       F       F       F       T
SPHERE_STDV                                     

28 NMR distance restraints 6 (X-Y)    :       0       0      0   0.000   0.000      0.0000       1.000
29 NMR distance restraints 7 (X-Y)    :       0       0      0   0.000   0.000      0.0000       1.000
30 Minimal distance restraints        :       0       0      0   0.000   0.000      0.0000       1.000
31 Non-bonded restraints              :       0       0      0   0.000   0.000      0.0000       1.000
32 Atomic accessibility restraints    :       0       0      0   0.000   0.000      0.0000       1.000
33 Atomic density restraints          :       0       0      0   0.000   0.000      0.0000       1.000
34 Absolute position restraints       :       0       0      0   0.000   0.000      0.0000       1.000
35 Dihedral angle difference restraint:       0       0      0   0.000   0.000      0.0000       1.000
36 GBSA implicit solvent potential    :       0       0      0   0.000   0.000      0.0000       1.000
37 EM density fitting potential       :       0       0      0   0.000   